In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import models, layers
from keras.utils import np_utils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

# Loading, exploring and visualization

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
sample_submission.head()

In [ ]:
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train_df.head()

In [ ]:
missing_value_counts = train_df.isnull().sum()
total_missing_values = missing_value_counts.sum()
total_missing_values

In [ ]:
print(train_df.info())

In [ ]:
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
test_df.head()

In [ ]:
print(test_df.info())

In [ ]:
X = train_df.drop("label", axis=1)
y = train_df.loc[:, "label"]

In [ ]:
plt.figure(figsize=(5,5))
plt.xticks(size=10)
sns.countplot(y, linewidth = 2, edgecolor = sns.color_palette("Set2"))
plt.title(" Label distribution of training dataset", fontdict={"color": "black", "fontsize": 15})
plt.show()

# Preprocessing

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(X, y, test_size=0.2)

In [ ]:
train_images = np.array(train_x)
train_images = train_images.reshape(len(train_images), 28,28,1)
train_images = train_images.astype("float32")
train_images = train_images / 255
train_labels = np.array(train_y)

In [ ]:
val_images = np.array(val_x)
val_images = val_images.reshape(len(val_images), 28,28,1)
val_images = val_images.astype("float32")
val_images = val_images / 255
val_labels = np.array(val_y)

In [ ]:
test_images = np.array(test_df)
test_images = test_images.reshape(len(test_images), 28,28,1)
test_images = test_images.astype("float32")
test_images = test_images / 255

In [ ]:
fig, axes = plt.subplots(5,5, figsize=(15, 15))
axes = axes.ravel()

for i in np.arange(0, 25):
    axes[i].imshow(train_images[i], cmap="Greys")
    axes[i].set_title("Digit = %s" % train_labels[i])
    axes[i].axis("off")
    plt.subplots_adjust(wspace=1)

# Model building

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), padding="same", activation="relu", input_shape=(28,28,1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation="softmax"))
model.summary()

In [ ]:
model.compile(optimizer="adam",
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

# Model training and validation

In [ ]:
batch_size = 264
history = model.fit(train_images, train_labels, epochs=10, batch_size=batch_size,
                    verbose=1,validation_data=(val_images, val_labels))

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ["loss", "val_loss"]].plot()
history_frame.loc[:, ["accuracy", "val_accuracy"]].plot()

# Predictions

In [ ]:
predictions = model.predict(test_images)
prediction_classes = np.argmax(predictions, axis=1)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
fig, axes = plt.subplots(5,5, figsize=(15,15))
axes= axes.ravel()

for i in np.arange(0,25):
    axes[i].imshow(test_images[i], cmap="Greys")
    axes[i].set_title("Predict = %s" % prediction_classes[i])
    axes[i].axis("off")
    plt.subplots_adjust(wspace=0.5)

# Submission

In [ ]:
output = pd.DataFrame({"ImageId": sample_submission.ImageId, "label": prediction_classes})
output.to_csv("My_submission", index=False)
output.head()